**1. Data Loading & Initial Exploration**

In [32]:
import pandas as pd
import numpy as np


# Load the dataset
try:
    df = pd.read_csv('../data/raw/visits.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'data/raw/visits.csv' not found.")
    print("Please generate the dataset first by running 'python scripts/data_generator.py' from the project root.")
    df = None

if df is not None:
    # --- Initial Data Exploration ---
    print("\n--- Dataframe Head ---")
    print(df.head())

    print("\n--- Dataframe Info ---")
    df.info()

    print("\n--- Missing Values ---")
    print(df.isnull().sum())

    print("\n--- Duplicate Visit IDs ---")
    print(f"Number of duplicate visit_ids: {df.duplicated(subset=['visit_id']).sum()}")

Dataset loaded successfully.

--- Dataframe Head ---
  visit_id patient_id nurse_id     visit_start_time       visit_end_time  \
0    v1000       p116      n25           1754323849           1754343349   
1    v1001       p110      n29           1757349713           1757351093   
2    v1002       p109      n23  11/23/2024 09:19 PM  11/23/2024 10:14 PM   
3    v1003       p132      n30  07/10/2025 10:18 PM  07/10/2025 11:24 PM   
4    v1004       p130      n26           1749741630           1749744990   

       service_type visit_location  \
0  General Check-up           Noth   
1        Wound Care           East   
2  Physical Therapy            Est   
3     Physiotherapy           West   
4  General Check-up            Est   

                                         nurse_notes  
0                                                NaN  
1  Drive interest respond end ask machine too or ...  
2  New only however enough mission wrong sport op...  
3                                        

**2. Data Cleaning & Preprocessing**

a. Handle Duplicate visit_id Records

The data generator intentionally creates duplicate visit_id entries. We'll remove them, keeping the first occurrence.


In [33]:
if df is not None:
    # Remove duplicate rows based on 'visit_id', keeping the first instance
    initial_rows = len(df)
    df.drop_duplicates(subset=['visit_id'], keep='first', inplace=True)
    print(f"Removed {initial_rows - len(df)} duplicate visit_id records.")
    print(f"Remaining records: {len(df)}")

Removed 9 duplicate visit_id records.
Remaining records: 191


b. Standardize Date/Time Formats

The visit_start_time and visit_end_time columns have mixed formats. We'll convert them to a standard datetime format.

In [34]:
if df is not None:
    # Standardize timestamp columns
    df['visit_start_time'] = pd.to_datetime(df['visit_start_time'], errors='coerce')
    df['visit_end_time'] = pd.to_datetime(df['visit_end_time'], errors='coerce')

    print("--- Data types after timestamp standardization ---")
    print(df[['visit_start_time', 'visit_end_time']].dtypes)

    # Check for any new NaNs created by conversion errors
    print("\n--- Missing timestamps after coercion ---")
    print(df[['visit_start_time', 'visit_end_time']].isnull().sum())

--- Data types after timestamp standardization ---
visit_start_time    datetime64[ns]
visit_end_time      datetime64[ns]
dtype: object

--- Missing timestamps after coercion ---
visit_start_time    40
visit_end_time      52
dtype: int64


/var/folders/m2/m8xt2gk918jcn12w3p2xldcm0000gn/T/ipykernel_87958/1492801177.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['visit_start_time'] = pd.to_datetime(df['visit_start_time'], errors='coerce')
/var/folders/m2/m8xt2gk918jcn12w3p2xldcm0000gn/T/ipykernel_87958/1492801177.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['visit_end_time'] = pd.to_datetime(df['visit_end_time'], errors='coerce')


**c. Handle Missing Values and Calculate Visit Duration**

Now that timestamps are standardized, we can calculate the visit duration. We'll then handle missing visit_end_time values. For this analysis, we will remove records with missing start or end times as duration is critical.

In [35]:
if df is not None:
    # Calculate visit duration in minutes
    df['visit_duration_minutes'] = (df['visit_end_time'] - df['visit_start_time']).dt.total_seconds() / 60

    # Handle missing timestamps
    # For this analysis, we'll remove rows where duration couldn't be calculated
    rows_before_drop = len(df)
    df.dropna(subset=['visit_start_time', 'visit_end_time'], inplace=True)
    print(f"Removed {rows_before_drop - len(df)} rows with missing start or end times.")

    # For missing nurse_notes, we'll fill them with a placeholder
    df['nurse_notes'].fillna('No Note Provided', inplace=True)
    print("\n--- Missing values after handling ---")
    print(df.isnull().sum())

Removed 52 rows with missing start or end times.

--- Missing values after handling ---
visit_id                  0
patient_id                0
nurse_id                  0
visit_start_time          0
visit_end_time            0
service_type              0
visit_location            0
nurse_notes               0
visit_duration_minutes    0
dtype: int64


/var/folders/m2/m8xt2gk918jcn12w3p2xldcm0000gn/T/ipykernel_87958/1299720180.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nurse_notes'].fillna('No Note Provided', inplace=True)


d. Address Outliers in Visit Durations

The data generator creates unrealistically short and long visits. We'll define reasonable bounds (e.g., 10 to 240 minutes) and remove records outside this range.

In [36]:
if df is not None:
    print("--- Visit Duration Statistics (Before Outlier Removal) ---")
    print(df['visit_duration_minutes'].describe())

    # Define reasonable bounds for visit duration
    min_duration = 10  # minutes
    max_duration = 240 # 4 hours

    # Filter out outliers
    rows_before_filter = len(df)
    df = df[(df['visit_duration_minutes'] >= min_duration) & (df['visit_duration_minutes'] <= max_duration)]
    print(f"\nRemoved {rows_before_filter - len(df)} records with outlier visit durations.")

    print("\n--- Visit Duration Statistics (After Outlier Removal) ---")
    print(df['visit_duration_minutes'].describe())

--- Visit Duration Statistics (Before Outlier Removal) ---
count    139.000000
mean      60.244604
std       58.619856
min        1.000000
25%       35.000000
50%       53.000000
75%       75.000000
max      552.000000
Name: visit_duration_minutes, dtype: float64

Removed 6 records with outlier visit durations.

--- Visit Duration Statistics (After Outlier Removal) ---
count    133.000000
mean      53.586466
std       20.411720
min       21.000000
25%       36.000000
50%       53.000000
75%       74.000000
max       90.000000
Name: visit_duration_minutes, dtype: float64


e. Clean and Standardize Categorical Variables

The service_type and visit_location columns contain typos and inconsistencies. We'll standardize them using mapping dictionaries.

In [37]:
if df is not None:
    # --- Clean 'service_type' ---
    service_type_mapping = {
        "Medication Administrtion": "Medication Administration",
        "WoundCare": "Wound Care",
        "Physiotherapy": "Physical Therapy",
        "General Checkup": "General Check-up"
    }
    df['service_type'] = df['service_type'].replace(service_type_mapping)
    # Also handle 'Unknown' from duplicate processing
    df = df[df['service_type'] != 'Unknown']


    # --- Clean 'visit_location' ---
    location_mapping = {
        "Noth": "North",
        "Suth": "South",
        "Est": "East",
        "Wst": "West"
    }
    df['visit_location'] = df['visit_location'].replace(location_mapping)

    print("--- Standardized Categorical Values ---")
    print("\nUnique Service Types:")
    print(df['service_type'].unique())
    print("\nUnique Visit Locations:")
    print(df['visit_location'].unique())

--- Standardized Categorical Values ---

Unique Service Types:
['Physical Therapy' 'Wound Care' 'Medication Administration' 'IV Therapy'
 'General Check-up']

Unique Visit Locations:
['East' 'West' 'North' 'South']


f. Extract Information from nurse_notes

We can use regular expressions to extract structured data like temperature and blood pressure, and flag notes containing keywords.

In [38]:
if df is not None:
    # --- Extract structured data from nurse_notes ---

    # Extract Temperature (e.g., "98.6F")
    df['temperature_f'] = df['nurse_notes'].str.extract(r'PATIENT_TEMP=(\d{2,3}\.\d)F').astype(float)

    # Extract Blood Pressure (e.g., "120/80")
    df['blood_pressure'] = df['nurse_notes'].str.extract(r'BP=(\d{2,3}/\d{2,3})')

    # --- Flag notes with keywords ---
    df['urgent_flag'] = df['nurse_notes'].str.contains(r'\*\*\*CHECK THIS\*\*\*|!!!', case=False, regex=True)

    print("\n--- Extracted data from nurse_notes ---")
    print(df[['nurse_notes', 'temperature_f', 'blood_pressure', 'urgent_flag']].head(10))

    print("\n--- Final Cleaned DataFrame ---")
    df.info()
    print(df.head())


--- Extracted data from nurse_notes ---
                                          nurse_notes  temperature_f  \
2   New only however enough mission wrong sport op...           99.5   
3                                    No Note Provided            NaN   
5   Minute suddenly no product much help from data...           97.3   
7   Call contain we start age say check success an...            NaN   
9   Morning huge campaign than policy authority da...            NaN   
10  Still box here everybody save message attentio...          100.0   
11                                   No Note Provided            NaN   
13  Marriage cause impact reach hit medical box si...            NaN   
14                                   No Note Provided            NaN   
15  Himself grow green party light everything past...            NaN   

   blood_pressure  urgent_flag  
2          140/80        False  
3             NaN        False  
5          128/81        False  
7             NaN        False  
9

**3. Save the Cleaned Data**

Finally, save the cleaned DataFrame to the data/processed/ directory for future analysis.

In [39]:
import os

if df is not None:
    # Define the output path
    output_path = '../data/processed/visits_cleaned.csv'
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save the cleaned dataframe
    df.to_csv(output_path, index=False)
    
    print(f"\nCleaned data saved to '{output_path}' with {len(df)} records.")


Cleaned data saved to '../data/processed/visits_cleaned.csv' with 133 records.
